In [1]:
import foolbox
import keras
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import PIL, os, shutil

Using TensorFlow backend.


In [9]:
filename = 'original.png'
filename = 'adverserial.png'
filename = 'overlay.png'
filename = './adverserial_frames/417.png'
keras.backend.set_learning_phase(0)
model = ResNet50(weights='imagenet')
preprocessing = (np.array([104, 116, 123]), 1)
fmodel = foolbox.models.KerasModel(model, bounds=(0, 255), preprocessing=(np.array([104, 116, 123]), 1))
img = image.load_img(filename, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
label = int(preds.argmax(axis=-1)[0])
print(decode_predictions(preds, top=2), label)

[[('n03891332', 'parking_meter', 0.19580378), ('n03447447', 'gondola', 0.14906652)]] 704


In [ ]:
y = image.img_to_array(img)
attack = foolbox.attacks.FGSM(fmodel)
adversarial = attack(y[:, :, ::-1], label)
matplotlib.image.imsave('original.png', y / 255)
matplotlib.image.imsave('adverserial.png', adversarial[:, :, ::-1] / 255)
difference = adversarial[:, :, ::-1] - y
matplotlib.image.imsave('difference.png', difference / abs(difference).max() * 0.2 + 0.5)

In [ ]:
%matplotlib inline
plt.figure()
plt.subplot(1, 3, 1)
plt.title('Original')
plt.imshow(y / 255)  # division by 255 to convert [0, 255] to [0, 1]
plt.axis('off')
plt.subplot(1, 3, 2)
plt.title('Adversarial')
plt.imshow(adversarial[:, :, ::-1] / 255)  # ::-1 to convert BGR to RGB
plt.axis('off')
plt.subplot(1, 3, 3)
plt.title('Difference')
difference = adversarial[:, :, ::-1] - y
plt.imshow(difference / abs(difference).max() * 0.2 + 0.5)
plt.axis('off')
plt.show()

In [13]:
try:
    shutil.rmtree('adverserial_frames')
    os.mkdir('adverserial_frames')
except:
    try:
        os.mkdir('adverserial_frames')
    except:
        pass
files = [f for f in os.listdir('./frames/') if os.path.isfile(os.path.join('./frames/', f))]
files_num = []
for f in files:
    files_num.append(int(f.split('.')[0]))
files_num = sorted(files_num)
files = files_num
imbg = PIL.Image.open('./frames/1.png').convert("RGBA")
imfg = PIL.Image.open('difference.png').convert("RGBA")
imbg_width, imbg_height = imbg.size
imfg_resized = imfg.resize((imbg_width, imbg_height), PIL.Image.ANTIALIAS)
for f in files:
    path = './frames/' + str(f) + '.png'
    imbg = PIL.Image.open(path).convert("RGBA")
    new_frame = PIL.Image.blend(imbg, imfg_resized, alpha=0.001)
    new_frame.save('./adverserial_frames/' + str(f) + '.png')